# Problem 1-2: Discounted Oversupply Optimization

## Problem Statement

Optimize planting decisions for 2024-2030 where:
- **Production up to expected sales** sold at normal price
- **Excess production** sold at 50% discount
- Market capacity based on 2023 actual production (stable demand assumption)

## Model Overview

**Two-stage MILP approach:**

1. **Stage 1:** Maximize total profit with discounted oversupply
2. **Stage 2:** Minimize planting fragmentation while retaining ≥95% of Stage 1 profit

**Key difference from Problem 1-1:** Piecewise revenue function requires binary variables and Big-M constraints.

## Piecewise Revenue Function

For crop $i$ in year $n$ with total yield $Y_{in}$:

$$R_i(Y_{in}) = \begin{cases}
p_i \cdot Y_{in} & \text{if } Y_{in} \leq Q_i \\
p_i \cdot Q_i + 0.5p_i \cdot (Y_{in} - Q_i) & \text{if } Y_{in} > Q_i
\end{cases}$$

Linearized using:
- $y^{norm}_{in}$: Quantity sold at normal price (≤ $Q_i$)
- $y^{excess}_{in}$: Quantity sold at 50% discount
- $\lambda_{in} \in \{0,1\}$: Oversupply indicator

In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

## 1. Data Loading

In [2]:
# Load datasets
sales = pd.read_csv('sales_volume_data.csv')
farmland = pd.read_excel('Attachment1_EN.xlsx', sheet_name='Existing Village Farmland')
crop_info = pd.read_excel('Attachment1_EN.xlsx', sheet_name='Village Crops')

# Clean data
farmland['Plot Type'] = farmland['Plot Type'].str.strip()
sales['Plot Type'] = sales['Plot Type'].str.strip()
sales['Crop Name'] = sales['Crop Name'].str.strip()

# Define scope
years = list(range(2024, 2031))
plots = farmland['Plot Name'].tolist()
crops = sales['Crop Name'].unique().tolist()

# Classify plots
single_season_types = ['Flat Dry Land', 'Terraced Field', 'Hillside Land']
single_plots = farmland[farmland['Plot Type'].isin(single_season_types)]['Plot Name'].tolist()

# Identify legumes
bean_types = ['Grain (Legumes)', 'Vegetable (Legumes)']
bean_crops = set(crop_info[crop_info['Crop Type'].isin(bean_types)]['Crop Name'])

# Season timeline for rotation
seasons_timeline = [(y, s) for y in years for s in [1, 2]]

print(f"✓ {len(crops)} crops, {len(plots)} plots, {len(years)} years")

✓ 41 crops, 54 plots, 7 years


## 2. Parameter Construction

### Plot Characteristics

$$A_j = \text{available land area for plot } j \text{ (mu)}$$

### Economic Parameters

$$y_{ij} = \text{yield per mu of crop } i \text{ on plot } j \text{ (jin/mu)}$$

$$c_{ij} = \text{production cost of crop } i \text{ on plot } j \text{ (yuan/mu)}$$

$$p_i = \text{selling price for crop } i \text{ (yuan/jin)}$$

$$Q_i = \text{expected annual sales for crop } i \text{ (jin)}$$

### Big-M Constants

For each crop $i$, compute maximum possible production:

$$M_i = \sum_j A_j \times \max_j y_{ij}$$

In [3]:
# Plot characteristics
Area = {row['Plot Name']: row['Plot Area (Mu)'] for _, row in farmland.iterrows()}
PlotType = {row['Plot Name']: row['Plot Type'] for _, row in farmland.iterrows()}

# Economic parameters
Yield, Cost, Price, ExpectedSales = {}, {}, {}, {}

for _, row in sales.iterrows():
    crop = row['Crop Name']
    ptype = row['Plot Type']
    matching_plots = [p for p in plots if PlotType[p] == ptype]
    
    for p in matching_plots:
        Yield[(crop, p)] = row['Yield_per_mu']
        Cost[(crop, p)] = row['Cost_per_mu']
    
    Price[crop] = row['Avg_Price']
    ExpectedSales[crop] = row['Expected_Sales_Volume']

# Big-M constants
M_big_per_crop = {}
for crop in crops:
    max_yield = max([Yield.get((crop, p), 0) for p in plots])
    M_big_per_crop[crop] = sum(Area.values()) * max_yield * 2

print(f"Parameters built: {len(Yield)} crop-plot pairs")
print(f"Total market capacity: {sum(ExpectedSales.values()):,.0f} jin/year")

Parameters built: 990 crop-plot pairs
Total market capacity: 2,833,300 jin/year


## 3. Constraint Functions

### 3.1 Agronomic Constraints

**C1. Variable Linking (Big-M):**
$$0.5 \cdot H_{ijns} \leq a_{ijns} \leq A_j \cdot H_{ijns}$$

**C2. Rotation:**
$$H_{ijns} + H_{ijn(s+1)} \leq 1$$

**C3. Land Capacity:**
- Single-season: $\sum_i a_{ijn1} \leq A_j$
- Multi-season: $\sum_i a_{ijns} \leq A_j, \; s \in \{1,2\}$

**C4. Legume Rotation:**
- Annual limit: $\sum_{i \in \mathcal{B}} \sum_s H_{ijns} \leq 1$
- 3-year minimum: $\sum_{\tau=n}^{n+2} \sum_s \sum_{i \in \mathcal{B}} H_{ij\tau s} \geq 1$

### 3.2 Piecewise Pricing Constraints

**C5. Production Decomposition:**
$$y^{norm}_{in} + y^{excess}_{in} = \sum_{j,s} a_{ijns} \cdot y_{ij}$$

**C6. Normal Sales Cap:**
$$y^{norm}_{in} \leq Q_i$$

**C7. Oversupply Logic (Big-M):**
$$\sum_{j,s} a_{ijns} \cdot y_{ij} - Q_i \leq M_i \cdot \lambda_{in}$$
$$y^{excess}_{in} \leq M_i \cdot \lambda_{in}$$

When $\lambda_{in}=0$: forces $y^{excess}_{in}=0$ (no oversupply)  
When $\lambda_{in}=1$: allows oversupply

### 3.3 Crop Diversification

**C8. Single Crop Limit:**
$$\sum_{j,s} a_{ijns} \leq 0.2 \times \sum_j A_j, \quad \forall i,n$$

In [4]:
def add_agronomic_constraints(model, a, H):
    """C1-C4: Basic agronomic constraints."""
    MIN_AREA = 0.5
    
    # C1: Variable linking
    for key in a:
        model.addConstr(a[key] <= Area[key[1]] * H[key])
        model.addConstr(a[key] >= MIN_AREA * H[key])
    
    # C2: Rotation
    for crop in crops:
        for plot in plots:
            for i in range(len(seasons_timeline) - 1):
                y1, s1 = seasons_timeline[i]
                y2, s2 = seasons_timeline[i + 1]
                k1, k2 = (crop, plot, y1, s1), (crop, plot, y2, s2)
                if k1 in H and k2 in H:
                    model.addConstr(H[k1] + H[k2] <= 1)
    
    # C3: Land capacity
    for plot in plots:
        for year in years:
            if plot in single_plots:
                model.addConstr(
                    gp.quicksum(a[k] for k in a if k[1]==plot and k[2]==year and k[3]==1) <= Area[plot]
                )
            else:
                for s in [1, 2]:
                    model.addConstr(
                        gp.quicksum(a[k] for k in a if k[1]==plot and k[2]==year and k[3]==s) <= Area[plot]
                    )
    
    # C4: Legume rotation
    for plot in plots:
        # Annual limit
        for year in years:
            model.addConstr(
                gp.quicksum(H[k] for k in H if k[0] in bean_crops and k[1]==plot and k[2]==year) <= 1
            )
        
        # 3-year minimum (plots ≥ 5 mu)
        if Area[plot] >= 5.0:
            for i in range(len(years) - 2):
                window = years[i:i + 3]
                model.addConstr(
                    gp.quicksum(H[k] for k in H if k[0] in bean_crops and k[1]==plot and k[2] in window) >= 1
                )

def add_pricing_constraints(model, a, Lambda, Excess, SoldNormal):
    """C5-C7: Piecewise pricing constraints."""
    for crop in crops:
        for year in years:
            M_big = M_big_per_crop[crop]
            total_prod = gp.quicksum(a[k] * Yield[(k[0], k[1])] for k in a if k[0]==crop and k[2]==year)
            
            # C5: Decomposition
            model.addConstr(SoldNormal[(crop, year)] + Excess[(crop, year)] == total_prod)
            
            # C6: Normal sales cap
            model.addConstr(SoldNormal[(crop, year)] <= ExpectedSales[crop])
            model.addConstr(SoldNormal[(crop, year)] <= total_prod)
            
            # C7: Oversupply logic (Big-M)
            model.addConstr(total_prod - ExpectedSales[crop] <= M_big * Lambda[(crop, year)])
            model.addConstr(Excess[(crop, year)] <= M_big * Lambda[(crop, year)])
            model.addConstr(Excess[(crop, year)] >= total_prod - ExpectedSales[crop])

def add_crop_limits(model, a):
    """C8: Single crop area limit (20%)."""
    MAX_RATIO = 0.2
    max_area = sum(Area.values()) * MAX_RATIO
    for crop in crops:
        for year in years:
            model.addConstr(
                gp.quicksum(a[k] for k in a if k[0]==crop and k[2]==year) <= max_area
            )

## 4. Stage 1: Maximize Profit

### Objective Function

$$\max Z = \sum_{i,n} \left[ p_i \cdot y^{norm}_{in} + 0.5p_i \cdot y^{excess}_{in} \right] - \sum_{i,j,n,s} c_{ij} \cdot a_{ijns}$$

where:
- First term: Revenue from normal-priced sales
- Second term: Revenue from discounted oversupply
- Third term: Total production cost

In [5]:
model_s1 = gp.Model("Stage1")
model_s1.setParam('OutputFlag', 0)
model_s1.setParam('MIPGap', 0.01)

# Create variables
a, H = {}, {}
for crop in crops:
    for plot in plots:
        if (crop, plot) not in Yield:
            continue
        for year in years:
            seasons = [1] if plot in single_plots else [1, 2]
            for s in seasons:
                key = (crop, plot, year, s)
                a[key] = model_s1.addVar(lb=0, ub=Area[plot], name=f"a_{crop}_{plot}_{year}_S{s}")
                H[key] = model_s1.addVar(vtype=GRB.BINARY, name=f"H_{crop}_{plot}_{year}_S{s}")

Lambda, Excess, SoldNormal = {}, {}, {}
for crop in crops:
    for year in years:
        Lambda[(crop, year)] = model_s1.addVar(vtype=GRB.BINARY, name=f"Lambda_{crop}_{year}")
        Excess[(crop, year)] = model_s1.addVar(lb=0, name=f"Excess_{crop}_{year}")
        SoldNormal[(crop, year)] = model_s1.addVar(lb=0, name=f"SoldNormal_{crop}_{year}")

model_s1.update()

# Objective function
revenue = gp.quicksum(
    SoldNormal[(c, y)] * Price[c] + Excess[(c, y)] * Price[c] * 0.5
    for c in crops for y in years
)
cost = gp.quicksum(a[k] * Cost[(k[0], k[1])] for k in a)
model_s1.setObjective(revenue - cost, GRB.MAXIMIZE)

# Add constraints
add_agronomic_constraints(model_s1, a, H)
add_pricing_constraints(model_s1, a, Lambda, Excess, SoldNormal)
add_crop_limits(model_s1, a)

model_s1.optimize()
P_star = model_s1.ObjVal

print(f"Stage 1 optimal profit: ¥{P_star:,.0f}")
print(f"Average per year: ¥{P_star/7:,.0f}")

Set parameter Username
Set parameter LicenseID to value 2735705
Academic license - for non-commercial use only - expires 2026-11-10
Stage 1 optimal profit: ¥98,462,192
Average per year: ¥14,066,027


## 5. Stage 2: Minimize Fragmentation

### Objective Function

$$\min Z_2 = \sum_{i,j,n,s} H_{ijns}$$

### Profit Constraint

$$Z \geq 0.95 \times Z_1^*$$

In [6]:
model_s2 = model_s1.copy()
model_s2.setParam('OutputFlag', 0)
model_s2.setParam('MIPGap', 0.10)

# Retrieve variables
a2 = {k: model_s2.getVarByName(f"a_{k[0]}_{k[1]}_{k[2]}_S{k[3]}") for k in a}
H2 = {k: model_s2.getVarByName(f"H_{k[0]}_{k[1]}_{k[2]}_S{k[3]}") for k in H}
Lambda2 = {k: model_s2.getVarByName(f"Lambda_{k[0]}_{k[1]}") for k in Lambda}
Excess2 = {k: model_s2.getVarByName(f"Excess_{k[0]}_{k[1]}") for k in Excess}
SoldNormal2 = {k: model_s2.getVarByName(f"SoldNormal_{k[0]}_{k[1]}") for k in SoldNormal}

# New objective
model_s2.setObjective(gp.quicksum(H2.values()), GRB.MINIMIZE)

# Profit constraint
revenue2 = gp.quicksum(
    SoldNormal2[(c, y)] * Price[c] + Excess2[(c, y)] * Price[c] * 0.5
    for c in crops for y in years
)
cost2 = gp.quicksum(a2[k] * Cost[(k[0], k[1])] for k in a2)
model_s2.addConstr(revenue2 - cost2 >= 0.95 * P_star)

model_s2.optimize()
final_profit = (revenue2 - cost2).getValue()

print(f"Stage 2 final profit: ¥{final_profit:,.0f} ({final_profit/P_star*100:.1f}%)")
print(f"Average per year: ¥{final_profit/7:,.0f}")

Stage 2 final profit: ¥93,546,105 (95.0%)
Average per year: ¥13,363,729


## 6. Extract Results

In [8]:
results = []
for k in a2:
    if a2[k].X > 0.01:
        crop, plot, year, season = k
        results.append({
            'Year': year,
            'Plot': plot,
            'Season': season,
            'Crop': crop,
            'Area': a2[k].X
        })

df_results = pd.DataFrame(results)
df_results = df_results.sort_values(['Year', 'Plot', 'Season'])

print(f"Extracted {len(results)} planting decisions")

Extracted 483 planting decisions


## 7. Oversupply Analysis

In [9]:
oversupply_data = []
for crop in crops:
    for year in years:
        if Lambda2[(crop, year)].X > 0.5:
            total = sum(a2[k].X * Yield[(k[0], k[1])] for k in a2 if k[0]==crop and k[2]==year)
            excess = Excess2[(crop, year)].X
            oversupply_data.append({
                'Crop': crop,
                'Year': year,
                'Total_Production': total,
                'Expected_Sales': ExpectedSales[crop],
                'Excess': excess,
                'Excess_Pct': excess / total * 100 if total > 0 else 0
            })

df_oversupply = pd.DataFrame(oversupply_data)

print(f"\nCrops with oversupply: {len(df_oversupply)}")
print(f"Total excess: {df_oversupply['Excess'].sum():,.0f} jin")
print(f"\nTop 5 oversupplied crops:")
print(df_oversupply.nlargest(5, 'Excess')[['Crop', 'Year', 'Excess', 'Excess_Pct']])


Crops with oversupply: 287
Total excess: 14,234,564 jin

Top 5 oversupplied crops:
         Crop  Year     Excess  Excess_Pct
196  Cucumber  2024  1287750.0   98.451835
197  Cucumber  2025  1287750.0   98.451835
198  Cucumber  2026  1287750.0   98.451835
199  Cucumber  2027  1287750.0   98.451835
200  Cucumber  2028  1287750.0   98.451835


## 8. Validation

In [10]:
print("=" * 60)
print("VALIDATION CHECKS")
print("=" * 60)

# Rotation violations
rotation_viol = 0
for i in range(len(seasons_timeline) - 1):
    y1, s1 = seasons_timeline[i]
    y2, s2 = seasons_timeline[i + 1]
    sn = df_results[(df_results['Year']==y1) & (df_results['Season']==s1)]
    sn1 = df_results[(df_results['Year']==y2) & (df_results['Season']==s2)]
    for _, row in sn.iterrows():
        if len(sn1[(sn1['Plot']==row['Plot']) & (sn1['Crop']==row['Crop'])]) > 0:
            rotation_viol += 1

# Legume violations
legume_viol = 0
for plot in plots:
    if Area[plot] < 5:
        continue
    for i in range(len(years) - 2):
        window = years[i:i + 3]
        data = df_results[(df_results['Plot']==plot) & (df_results['Year'].isin(window))]
        if len(data[data['Crop'].isin(bean_crops)]) < 1:
            legume_viol += 1

# Piecewise linearization verification
piecewise_errors = 0
for crop in list(crops)[:5]:
    for year in [2024]:
        total = sum(a2[k].X * Yield[(k[0], k[1])] for k in a2 if k[0]==crop and k[2]==year)
        sold_normal = SoldNormal2[(crop, year)].X
        excess = Excess2[(crop, year)].X
        
        if abs(sold_normal + excess - total) > 1:
            piecewise_errors += 1

print(f"Rotation violations: {rotation_viol}")
print(f"Legume violations: {legume_viol}")
print(f"Piecewise errors: {piecewise_errors}")

if rotation_viol == 0 and legume_viol == 0 and piecewise_errors == 0:
    print("\n✓ All constraints satisfied")
else:
    print(f"\n⚠ Found {rotation_viol + legume_viol + piecewise_errors} violations")

print("=" * 60)

VALIDATION CHECKS
Rotation violations: 0
Legume violations: 0
Piecewise errors: 0

✓ All constraints satisfied


## 9. Fill Excel Template

In [ ]:
import openpyxl

wb = openpyxl.load_workbook('result1_2_EN.xlsx')

# Get crop columns
ws_sample = wb['2024']
crop_to_col = {}
for col_idx in range(3, ws_sample.max_column + 1):
    crop_name = ws_sample.cell(1, col_idx).value
    if crop_name:
        crop_to_col[crop_name] = col_idx

# Build plot-row mapping
def get_plot_rows(ws):
    season1, season2 = {}, {}
    current_season = None
    for row_idx in range(2, 85):
        marker = ws.cell(row_idx, 1).value
        plot = ws.cell(row_idx, 2).value
        if marker == "Season 1":
            current_season = 1
        elif marker == "Season 2":
            current_season = 2
        if plot and current_season == 1:
            season1[plot] = row_idx
        elif plot and current_season == 2:
            season2[plot] = row_idx
    return season1, season2

# Fill each year
for year in years:
    ws = wb[str(year)]
    season1_map, season2_map = get_plot_rows(ws)
    valid_rows = set(season1_map.values()) | set(season2_map.values())
    
    for row_idx in valid_rows:
        for col_idx in range(3, ws.max_column + 1):
            ws.cell(row_idx, col_idx).value = 0
    
    year_data = df_results[df_results['Year'] == year]
    for _, row in year_data.iterrows():
        plot, season, crop, area = row['Plot'], int(row['Season']), row['Crop'], round(row['Area'], 2)
        row_idx = season1_map.get(plot) if season == 1 else season2_map.get(plot)
        col_idx = crop_to_col.get(crop)
        if row_idx and col_idx:
            ws.cell(row_idx, col_idx).value = area

wb.save('result1_2_OUTPUT.xlsx')
print("Saved to result1_2_OUTPUT.xlsx")

✓ Saved to result1_2_OUTPUT.xlsx
